In [124]:
import json
import pandas as pd
import fitz
import numpy

In [125]:
budget = pd.read_csv('budget_resource.csv')
budget = budget[budget.interested & ~budget.interested.isna()]

In [126]:
budget

,fisical_year,name,path,interested,url,start,end,Unnamed: 7
6,61.0,สำนักงานเลขานุการสภากรุงเทพมหานคร,61/A20171003162231.pdf,True,3,NaN,NaN,NaN
7,61.0,สำนักงานเลขานุการผู้ว่าราชการกรุงเทพมหานคร,61/A20171003162300.pdf,True,3,NaN,NaN,NaN
8,61.0,สำนักงานคณะกรรมการข้าราชการกรุงเทพมหานคร,61/A20171003162330.pdf,True,3,NaN,NaN,NaN
9,61.0,สำนักปลัดกรุงเทพมหานคร,61/A20171003162350.pdf,True,3,NaN,NaN,NaN
10,61.0,สำนักการแพทย์,61/A20171003162552.pdf,True,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
389,65.0,73. สำนักงานเขตทวีวัฒนา,65/65073.pdf,True,http://office.bangkok.go.th/budd/main/../main/...,26.0,NaN,NaN
390,65.0,74. สำนักงานเขตทุ่งครุ,65/65074.pdf,True,http://office.bangkok.go.th/budd/main/../main/...,30.0,NaN,NaN
391,65.0,75. สำนักงานเขตบางบอน,65/65075.pdf,True,http://office.bangkok.go.th/budd/main/../main/...,28.0,NaN,NaN
392,65.0,76. มหาวิทยาลัยนวมินทราธิราช,65/65076.pdf,True,http://office.bangkok.go.th/budd/main/../main/...,4.0,NaN,NaN


In [119]:
def get_first_page(document):
    if isinstance(document, str):
        document = fitz.open(document)
    assert isinstance(document, fitz.fitz.Document)
    
    def is_vert_line(rect):
        return int(rect.y1) - int(rect.y0)
    
    found_graphic = False
    for page in document:
        found_vert_line = False
        for g in page.get_drawings():
            if is_vert_line(g['rect']):
                found_vert_line = True; break
        if found_vert_line:
            found_graphic = True
        elif found_graphic:
            return page.number
    return -100

In [120]:
start=budget[['path','start']].apply(
    lambda doc: get_first_page('../pdf/'+doc.path)+1 if numpy.isnan(doc.start) else doc.start,
    axis=1
)

In [123]:
budget.assign(auto_start=start).to_csv('budget-toc.csv')

# Use first section
for older format we can use beginning page for data extraction

In [137]:
def get_end_page(document):
    if isinstance(document, str):
        document = fitz.open(document)
    assert isinstance(document, fitz.fitz.Document)
    
    def is_vert_line(rect):
        return int(rect.y1) - int(rect.y0)
    
    for page in document:
        for g in page.get_drawings():
            if is_vert_line(g['rect']):
                return page.number
    return -100

In [138]:
end=budget[['path','start']].apply(
    lambda doc: get_end_page('../pdf/'+doc.path)+1 if numpy.isnan(doc.start) else -1,
    axis=1
)

In [141]:
sum(end == -99)

1

In [142]:
budget.assign(end=end).to_csv('budget-toc-end-page.csv')